In [1]:
# imports
import pymysql
#pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from sqlalchemy.types import *
from sqlalchemy_utils import database_exists
import pandas as pd
import numpy as np


In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
#print(basics)

In [3]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [4]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,264
2,tt0000003,6.5,1863
3,tt0000004,5.5,177
4,tt0000005,6.2,2649


In [7]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [8]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36951101 entries, 0 to 36951100
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


# Akas
     replacing \N with nan
     filtering US only movies

In [9]:
akas.replace({'\\N':np.nan}, inplace=True)

In [10]:
#akas = akas.loc[akas["region"] != 'US']
#akas.drop(akas.loc[akas["region"] != 'US'], inplace=True)
akas.drop(akas[akas['region'] != 'US'].index, inplace = True)

In [11]:
print(akas)

            titleId  ordering                                      title  \
5         tt0000001         6                                 Carmencita   
14        tt0000002         7                     The Clown and His Dogs   
33        tt0000005        10                           Blacksmith Scene   
36        tt0000005         1                        Blacksmithing Scene   
41        tt0000005         6                        Blacksmith Scene #1   
...             ...       ...                                        ...   
36950627  tt9916560         1  March of Dimes Presents: Once Upon a Dime   
36950697  tt9916620         1                          The Copeland Case   
36950786  tt9916702         1              Loving London: The Playground   
36950829  tt9916756         1                   Pretty Pretty Black Girl   
36950845  tt9916764         1                                         38   

         region language        types             attributes isOriginalTitle  
5       

# Basics

In [12]:
keepers =basics['tconst'].isin(akas['titleId'])
keepers

0            True
1            True
2           False
3           False
4            True
            ...  
10103805    False
10103806    False
10103807    False
10103808    False
10103809    False
Name: tconst, Length: 10103810, dtype: bool

In [13]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
...,...,...,...,...,...,...,...,...,...
10103671,tt9916560,tvMovie,March of Dimes Presents: Once Upon a Dime,March of Dimes Presents: Once Upon a Dime,0,1963,\N,58,Family
10103700,tt9916620,movie,The Copeland Case,The Copeland Case,0,\N,\N,\N,Drama
10103738,tt9916702,short,Loving London: The Playground,Loving London: The Playground,0,\N,\N,\N,"Drama,Short"
10103761,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,0,2019,\N,\N,Short


In [14]:
basics.replace({'\\N':np.nan}, inplace=True)

In [15]:

basics.drop(basics[basics['runtimeMinutes'].isnull()].index, inplace = True)

In [16]:
#import math
print(basics)
#print(math.NaN)

             tconst titleType                                 primaryTitle  \
0         tt0000001     short                                   Carmencita   
1         tt0000002     short                       Le clown et ses chiens   
4         tt0000005     short                             Blacksmith Scene   
5         tt0000006     short                            Chinese Opium Den   
6         tt0000007     short  Corbett and Courtney Before the Kinetograph   
...             ...       ...                                          ...   
10103503  tt9916214     short                              Drown the Clown   
10103523  tt9916254     video                         Big Tit Cream Pie 32   
10103569  tt9916348     video                        Ancient World Exposed   
10103576  tt9916362     movie                                        Coven   
10103671  tt9916560   tvMovie    March of Dimes Presents: Once Upon a Dime   

                                        originalTitle isAdult s

In [17]:
basics.drop(basics[basics['genres'].isnull()].index, inplace = True)

In [18]:
print(basics)

             tconst titleType                                 primaryTitle  \
0         tt0000001     short                                   Carmencita   
1         tt0000002     short                       Le clown et ses chiens   
4         tt0000005     short                             Blacksmith Scene   
5         tt0000006     short                            Chinese Opium Den   
6         tt0000007     short  Corbett and Courtney Before the Kinetograph   
...             ...       ...                                          ...   
10103503  tt9916214     short                              Drown the Clown   
10103523  tt9916254     video                         Big Tit Cream Pie 32   
10103569  tt9916348     video                        Ancient World Exposed   
10103576  tt9916362     movie                                        Coven   
10103671  tt9916560   tvMovie    March of Dimes Presents: Once Upon a Dime   

                                        originalTitle isAdult s

In [19]:
basics.drop(basics[basics['titleType'] != 'movie'].index, inplace = True)

In [20]:
print(basics)

             tconst titleType                    primaryTitle  \
8         tt0000009     movie                      Miss Jerry   
144       tt0000147     movie   The Corbett-Fitzsimmons Fight   
570       tt0000574     movie     The Story of the Kelly Gang   
587       tt0000591     movie                The Prodigal Son   
672       tt0000679     movie  The Fairylogue and Radio-Plays   
...             ...       ...                             ...   
10103165  tt9915436     movie               Vida em Movimento   
10103343  tt9915872     movie            The Last White Witch   
10103483  tt9916170     movie                   The Rehearsal   
10103492  tt9916190     movie                       Safeguard   
10103576  tt9916362     movie                           Coven   

                           originalTitle isAdult startYear endYear  \
8                             Miss Jerry       0      1894     NaN   
144        The Corbett-Fitzsimmons Fight       0      1897     NaN   
570      

In [21]:
basics['startYear'].value_counts

<bound method IndexOpsMixin.value_counts of 8           1894
144         1897
570         1906
587         1907
672         1908
            ... 
10103165    2019
10103343    2019
10103483    2019
10103492    2020
10103576    2020
Name: startYear, Length: 204586, dtype: object>

In [22]:
basics.isna().sum()
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204586 entries, 8 to 10103576
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          204586 non-null  object
 1   titleType       204586 non-null  object
 2   primaryTitle    204586 non-null  object
 3   originalTitle   204586 non-null  object
 4   isAdult         204586 non-null  object
 5   startYear       201010 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  204586 non-null  object
 8   genres          204586 non-null  object
dtypes: object(9)
memory usage: 15.6+ MB


In [23]:
basics['startYear'] = basics['startYear'].astype('float')

In [24]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204586 entries, 8 to 10103576
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          204586 non-null  object 
 1   titleType       204586 non-null  object 
 2   primaryTitle    204586 non-null  object 
 3   originalTitle   204586 non-null  object 
 4   isAdult         204586 non-null  object 
 5   startYear       201010 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  204586 non-null  object 
 8   genres          204586 non-null  object 
dtypes: float64(1), object(8)
memory usage: 15.6+ MB


In [25]:
basics.drop(basics[basics['startYear'] <= 2000].index, inplace = True)
basics.drop(basics[basics['startYear'] >= 2021].index, inplace = True)
#basics.loc[(basics['startYear'] < 2000) & (basics['startYear'] < 2021 )]
#basics.loc[(basics['startYear'] > 2021) & (basics['startYear'] < 2000)]
#basics.loc[(basics['startYear'] < 2000)]

In [26]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
81513,tt0083324,movie,The White Lions,The White Lions,0,NaN,NaN,97,Drama
...,...,...,...,...,...,...,...,...,...
10103165,tt9915436,movie,Vida em Movimento,Vida em Movimento,0,2019.0,NaN,70,Documentary
10103343,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10103483,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10103492,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [27]:
basics.drop(basics[basics['genres'] == 'Documentary'].index, inplace = True)

In [28]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
81513,tt0083324,movie,The White Lions,The White Lions,0,NaN,NaN,97,Drama
...,...,...,...,...,...,...,...,...,...
10102949,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10103343,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10103483,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10103492,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


# Rankings

In [29]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1990
1,tt0000002,5.8,264
2,tt0000003,6.5,1863
3,tt0000004,5.5,177
4,tt0000005,6.2,2649


In [30]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340967 entries, 0 to 1340966
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1340967 non-null  object 
 1   averageRating  1340967 non-null  float64
 2   numVotes       1340967 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.7+ MB


In [35]:
new_ratings = basics['tconst'].isin(ratings['tconst'])
new_ratings

34800        True
61112        True
67486        True
67664        True
81513       False
            ...  
10102949     True
10103343     True
10103483     True
10103492     True
10103576     True
Name: tconst, Length: 90310, dtype: bool

In [36]:
basics_new = basics[new_ratings]
basics_new

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86791,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
10102949,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
10103343,tt9915872,movie,The Last White Witch,Boku no kanojo wa mahoutsukai,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
10103483,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
10103492,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [41]:
## Save current dataframe to file.
basics_new.to_csv("C:/Users/ROSA/Documents/Data/title_basics.csv.gz",compression='gzip',index=False)

In [42]:
basics_new = pd.read_csv("C:/Users/ROSA/Documents/Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34800,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61112,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67486,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
67664,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
81513,tt0083324,movie,The White Lions,The White Lions,0,NaN,NaN,97,Drama
